## Desanidar y normalizar valores de movies
En esta parte desanidamos y normalizamos las columnas de movies_dataset.csv

In [2]:
import pandas as pd
from pandas import json_normalize
import ast
import json
import os
import zipfile
from ast import literal_eval

data_movies = pd.read_csv('D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Datasets\movies_dataset.csv')
data_movies.rename(columns={'id':'movie_id'}, inplace=True) #Cambiamos el id a movie_id para poder usarlo como primary key
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   movie_id               45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

C:\Users\Losse\AppData\Local\Temp\ipykernel_24036\2065980286.py:9: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_movies = pd.read_csv('D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Datasets\movies_dataset.csv')


a. Columna: belongs_to_collection

In [3]:
# Segun el diccionario entregado nos concentramos en: belongs_to_collection
data_movies_collection = pd.DataFrame(data_movies[['belongs_to_collection','movie_id']]).dropna()
data_movies_collection.head()

,belongs_to_collection,movie_id
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",862
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",15602
4,"{'id': 96871, 'name': 'Father of the Bride Col...",11862
9,"{'id': 645, 'name': 'James Bond Collection', '...",710
12,"{'id': 117693, 'name': 'Balto Collection', 'po...",21032


In [4]:
# tercero se normaliza la columna
data_movies_collection=data_movies_collection.set_index('movie_id')

# Reemplazar NaN en la columna 'belongs_to_collection' por un diccionario predeterminado
data_movies_collection['belongs_to_collection'] = data_movies_collection['belongs_to_collection'].apply(lambda x: {'id': 0, 'name': 'Desconocido', 'poster_path': 'N/A', 'backdrop_path': 'N/A'} if pd.isna(x) else x)

# Convierte de string a diccionario
data_movies_collection['belongs_to_collection'] = data_movies_collection['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Desanida la columna 'belongs_to_collection'
data_movies_collection_unnested = json_normalize(data_movies_collection['belongs_to_collection']).set_index(data_movies_collection.index)
#data_movies_collection_unnested = pd.json_normalize(data_movies_collection['belongs_to_collection']).set_index(data_movies_collection.index)

data_movies_collection_unnested.head()

,id,name,poster_path,backdrop_path
movie_id,,,,
862,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
15602,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
11862,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
710,645.0,James Bond Collection,/HORpg5CSkmeQlAolx3bKMrKgfi.jpg,/6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
21032,117693.0,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg


In [5]:
# Cambiamos el nombre de las columnas
data_movies_collection_unnested.rename(columns = {'id':'franquicia_id','name':'franquicia'}, inplace = True)
data_movies_collection_unnested.reset_index(inplace=True) # mueve el indice a una columna
data_movies_collection_unnested.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4494 entries, 0 to 4493
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       4494 non-null   object 
 1   franquicia_id  4491 non-null   float64
 2   franquicia     4491 non-null   object 
 3   poster_path    3948 non-null   object 
 4   backdrop_path  3263 non-null   object 
dtypes: float64(1), object(4)
memory usage: 175.7+ KB


b. Columna: production_companies

In [6]:
# nos concentramos en: production_companies
data_movies_companies=pd.DataFrame(data_movies[['production_companies','movie_id']]).dropna()
data_movies_companies.head()

,production_companies,movie_id
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]",862
1,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",8844
2,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",15602
3,[{'name': 'Twentieth Century Fox Film Corporat...,31357
4,"[{'name': 'Sandollar Productions', 'id': 5842}...",11862


In [7]:
data_movies_companies['production_companies']=data_movies_companies['production_companies'].apply(literal_eval)
data_movies_companies.head()

,production_companies,movie_id
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]",862
1,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",8844
2,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",15602
3,[{'name': 'Twentieth Century Fox Film Corporat...,31357
4,"[{'name': 'Sandollar Productions', 'id': 5842}...",11862


In [8]:
# segundo, expandimos la columna para que queden diccionarios anidados (deja de ser una lista de diccionarios)
data_movies_companies_exploded = data_movies_companies.explode('production_companies', ignore_index=True)
data_movies_companies_exploded.head(2)

,production_companies,movie_id
0,"{'name': 'Pixar Animation Studios', 'id': 3}",862
1,"{'name': 'TriStar Pictures', 'id': 559}",8844


In [9]:
# tercero se normaliza la columna
data_movies_companies_exploded=data_movies_companies_exploded.set_index('movie_id')
data_movies_companies_exploded_unnested = pd.json_normalize(data_movies_companies_exploded['production_companies']).set_index(data_movies_companies_exploded.index)
data_movies_companies_exploded_unnested.head(2)

,name,id
movie_id,,
862,Pixar Animation Studios,3.0
8844,TriStar Pictures,559.0


In [10]:
# se finaliza el df con el cambio de nombre de columna.
data_movies_companies_exploded_unnested.rename(columns = {'id':'productora_id','name':'productora'}, inplace = True)
data_movies_companies_exploded_unnested.reset_index(inplace=True) # mueve le indice a una columna
data_movies_companies_exploded_unnested.head(2)

,movie_id,productora,productora_id
0,862,Pixar Animation Studios,3.0
1,8844,TriStar Pictures,559.0


c. Columna: genres

In [11]:
# nos concentramos en:  genres 
data_movies_genres=pd.DataFrame(data_movies[['genres','movie_id']]).dropna()
data_movies_genres.head(2)

,genres,movie_id
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844


In [12]:
# replicamos los pasos para el caso de  lista de diccionario
data_movies_genres['genres']=data_movies_genres['genres'].apply(literal_eval)
data_movies_genres_exploded = data_movies_genres.explode('genres', ignore_index=True)
data_movies_genres_exploded=data_movies_genres_exploded.set_index('movie_id')
data_movies_genres_exploded_unnested = pd.json_normalize(data_movies_genres_exploded['genres']).set_index(data_movies_genres_exploded.index)
data_movies_genres_exploded_unnested.rename(columns = {'id':'genres_id','name':'genres'},inplace = True)
data_movies_genres_exploded_unnested.reset_index(inplace=True) # mueve le indice a una columna
data_movies_genres_exploded_unnested.head(2)


,movie_id,genres_id,genres
0,862,16.0,Animation
1,862,35.0,Comedy


In [13]:
data_movies_countries = pd.DataFrame(data_movies[['production_countries', 'movie_id']]).dropna()
data_movies_countries['production_countries'] = data_movies_countries['production_countries'].apply(literal_eval)
data_movies_countries_exploded = data_movies_countries.explode('production_countries')
data_movies_countries_exploded_unnested = pd.json_normalize(data_movies_countries_exploded['production_countries']).set_index(data_movies_countries_exploded.movie_id)
data_movies_countries_exploded_unnested.reset_index(inplace=True)
data_movies_countries_exploded_unnested.rename(columns={'iso_3166_1':'pais_isocode','name' : 'pais_name'},inplace=True)
data_movies_countries_exploded_unnested.head(2)

,movie_id,pais_isocode,pais_name
0,862,US,United States of America
1,8844,US,United States of America


In [14]:
data_movies_languages = pd.DataFrame(data_movies[['spoken_languages', 'movie_id']]).dropna()
data_movies_languages['spoken_languages']= data_movies_languages['spoken_languages'].apply(literal_eval)
data_movies_languages_exploded = data_movies_languages.explode('spoken_languages')
data_movies_languages_exploded_unnested = pd.json_normalize(data_movies_languages_exploded['spoken_languages']).set_index(data_movies_languages_exploded.movie_id)
data_movies_languages_exploded_unnested.reset_index(inplace=True)
data_movies_languages_exploded_unnested.rename(
    columns={'iso_639_1':'spoken_languages_isocode','name' : 'spoken_languages_name'},
    inplace=True
    )
data_movies_languages_exploded_unnested.head(2)

,movie_id,spoken_languages_isocode,spoken_languages_name
0,862,en,English
1,8844,en,English


In [15]:
# unir los dataframes
data_movies_normalized = pd.merge(
    pd.merge(pd.merge(pd.merge(data_movies_collection_unnested,
    data_movies_companies_exploded_unnested,
    on=['movie_id']),
    data_movies_genres_exploded_unnested,
    on=['movie_id']),
    data_movies_countries_exploded_unnested,
    on=['movie_id']),
    data_movies_languages_exploded_unnested,
    on=['movie_id']
    )
data_movies_normalized.head(2)

,movie_id,franquicia_id,franquicia,poster_path,backdrop_path,productora,productora_id,genres_id,genres,pais_isocode,pais_name,spoken_languages_isocode,spoken_languages_name
0,862,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,Pixar Animation Studios,3.0,16.0,Animation,US,United States of America,en,English
1,862,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,Pixar Animation Studios,3.0,35.0,Comedy,US,United States of America,en,English


In [16]:
data_movies_final=data_movies_normalized[
    data_movies_normalized.columns[~data_movies_normalized.columns.str.contains("_path")] # quitamos las NO contienen "_path"
    ]
data_movies_final.to_csv(os.path.join("D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Movies\Pipeline","data_movies_normalizada.csv"))
data_movies_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55897 entries, 0 to 55896
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   movie_id                  55897 non-null  object 
 1   franquicia_id             55897 non-null  float64
 2   franquicia                55897 non-null  object 
 3   productora                53879 non-null  object 
 4   productora_id             53879 non-null  float64
 5   genres_id                 55754 non-null  float64
 6   genres                    55754 non-null  object 
 7   pais_isocode              55285 non-null  object 
 8   pais_name                 55285 non-null  object 
 9   spoken_languages_isocode  55728 non-null  object 
 10  spoken_languages_name     55728 non-null  object 
dtypes: float64(3), object(8)
memory usage: 4.7+ MB
